In [1]:
import cv2
import mediapipe as mp
import pandas as pd
import re

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands
mp_holistic = mp.solutions.holistic

df = pd.DataFrame()

In [2]:
# Regular expression pattern to extract coordinates
pattern = r'x: ([-+]?[0-9]*\.?[0-9]+)\ny: ([-+]?[0-9]*\.?[0-9]+)\nz: ([-+]?[0-9]*\.?[0-9]+)'

def get_coordinates(input_string):
    # Extract coordinates using regex
    matches = re.findall(pattern, input_string)

    # Convert the extracted coordinates to a list of dictionaries
    coordinates = [{'x': float(match[0]), 'y': float(match[1]), 'z': float(match[2])} for match in matches]
    return coordinates

In [3]:
import os
path = os.getcwd()

In [4]:
def extract_video(video, target, sequence_id):
    global df
    # For webcam input:
    cap = cv2.VideoCapture(video)
    results = None
    with mp_holistic.Holistic(
        model_complexity=0,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as hands:

        while cap.isOpened():
            success, image = cap.read()
            if not success:
                print("Ignoring empty camera frame. Or video finished. ")
                # If loading a video, use 'break' instead of 'continue'.
                break

            results = hands.process(image)

            # # Draw the hand annotations on the image.
            image.flags.writeable = True
            # image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            new_row = {}
            if (results.right_hand_landmarks):
                array = get_coordinates(str(results.right_hand_landmarks.landmark))
                for i in range(len(array)):
                    new_row[f'x_right_hand_{i}'] = array[i]["x"]
                    new_row[f'y_right_hand_{i}'] = array[i]["y"]
                    new_row["target"] = target
                    new_row["sequence_id"] = sequence_id

                mp_drawing.draw_landmarks(
                    image,
                    results.right_hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing_styles.get_default_hand_landmarks_style(),
                    mp_drawing_styles.get_default_hand_connections_style())
            
            if (results.left_hand_landmarks):
                array = get_coordinates(str(results.left_hand_landmarks.landmark))
                for i in range(len(array)):
                    new_row[f'x_left_hand_{i}'] = array[i]["x"]
                    new_row[f'y_left_hand_{i}'] = array[i]["y"]
                    new_row["target"] = target
                    new_row["sequence_id"] = sequence_id

                mp_drawing.draw_landmarks(
                    image,
                    results.left_hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing_styles.get_default_hand_landmarks_style(),
                    mp_drawing_styles.get_default_hand_connections_style())
            
            if (new_row):
                df = df.append(new_row, ignore_index=True)    
            # Flip the image horizontally for a selfie-view display.
            cv2.imshow('MediaPipe Hands', cv2.flip(image, 1))
            if cv2.waitKey(5) & 0xFF == 27:
              break

    cap.release()
    cv2.destroyAllWindows()

In [5]:
import glob, os
os.chdir("D:\Documents\Trabajo\megaproyecto_model\data/asl_videos")

sequence_id = 0
for file in glob.glob("*.mp4"):
    sequence_id += 1
    name = " ".join(file.split(" ")[:-1])
    print(name)
    extract_video(f'D:\Documents\Trabajo\megaproyecto_model\data/asl_videos/{file}', name, sequence_id)

bathroom
Ignoring empty camera frame. Or video finished. 
bathroom
Ignoring empty camera frame. Or video finished. 
bathroom
Ignoring empty camera frame. Or video finished. 
bathroom
Ignoring empty camera frame. Or video finished. 
bathroom
Ignoring empty camera frame. Or video finished. 
cat
Ignoring empty camera frame. Or video finished. 
cat
Ignoring empty camera frame. Or video finished. 
cat
Ignoring empty camera frame. Or video finished. 
cat
Ignoring empty camera frame. Or video finished. 
dog
Ignoring empty camera frame. Or video finished. 
dog
Ignoring empty camera frame. Or video finished. 
dog
Ignoring empty camera frame. Or video finished. 
dog
Ignoring empty camera frame. Or video finished. 
eat food
Ignoring empty camera frame. Or video finished. 
eat food
Ignoring empty camera frame. Or video finished. 
eat food
Ignoring empty camera frame. Or video finished. 
eat food
Ignoring empty camera frame. Or video finished. 
father
Ignoring empty camera frame. Or video finished.

In [17]:
df['sequence_id'] = df['sequence_id'].astype(int)

In [18]:
df.head()

,sequence_id,target,x_right_hand_0,x_right_hand_1,x_right_hand_10,x_right_hand_11,x_right_hand_12,x_right_hand_13,x_right_hand_14,x_right_hand_15,...,y_left_hand_19,y_left_hand_2,y_left_hand_20,y_left_hand_3,y_left_hand_4,y_left_hand_5,y_left_hand_6,y_left_hand_7,y_left_hand_8,y_left_hand_9
0,1,bathroom,0.265968,0.292575,0.288380,0.292631,0.294420,0.267287,0.274185,0.278704,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,bathroom,0.261926,0.285769,0.270102,0.277428,0.281503,0.252827,0.257510,0.266150,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,bathroom,0.261108,0.279633,0.253392,0.264042,0.271299,0.240579,0.244216,0.256161,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,bathroom,0.264250,0.275873,0.238741,0.254208,0.264493,0.231736,0.232600,0.247959,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,bathroom,0.270280,0.274816,0.231373,0.249087,0.260723,0.226887,0.228234,0.246714,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
dir_data = "D:\Documents\Trabajo\megaproyecto_model" + "/data.csv"
df.to_csv(dir_data, index=False)

In [42]:
validation_path = "D:\Documents\Trabajo\megaproyecto_model" + "/validation.csv"

import random
grouped = df.groupby('target')
selected_validation_ids = []
for group_name, group_indices in grouped.groups.items():
    ids = list(set(df.loc[group_indices]['sequence_id'].values))
    selected_validation_ids.append(random.choice(ids))

validation_df = df[df['sequence_id'].isin(selected_validation_ids)][["sequence_id", "target"]]
validation_df.to_csv(validation_path, index=False)

In [43]:
train_path = "D:\Documents\Trabajo\megaproyecto_model" + "/train.csv"
train_df = df[~df['sequence_id'].isin(selected_validation_ids)][["sequence_id", "target"]]
train_df.to_csv(train_path, index=False)